#  EDA-3. Проектирование признаков 

проектировании признаков (Feature Engineering). Этот процесс также называют конструированием признаков или разработкой признаков.

На этапе проектирования признаков специалисты по данным преследуют такие же цели, как на этапе сбора данных — сконструировать как можно больше признаков, чтобы далее отобрать из них лучшие.

## 2. Создание признаков

Проектирование признаков — самый творческий процесс во всём цикле жизни создания модели машинного обучения. 

Он включает в себя несколько этапов: 

    создание признаков;
    преобразование признаков;
    отбор признаков.

Проектирование признаков необходимо для улучшения качества будущей модели: в процессе создания признаков появляются новые, возможно, более качественные, чем исходные.

Новые признаки могут быть сконструированы двумя способами: 

    1 с помощью внешних источников данных;
    2 из существующего набора данных.

Новые признаки можно сконструировать из уже существующего набора данных несколькими способами: 

   1 разбор категорий; 
   2 разбор даты и времени;
   3 разбор числовых признаков;
   4 разбор текста.

